In [2]:
import math
import os
import pandas as pd
from itertools import combinations
import scipy.cluster.hierarchy as sch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import hypergeom
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import pdist, squareform
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
import xlsxwriter


In [3]:
# Specify the directory path where your files are located
directory_path = 'eQTLs'

# Initialize an empty list to store DataFrames
df_list = []

# Initialize an empty list to store disorder names
disorder_names = []

# Iterate through files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):  # Process only .txt files
        # Extract disorder name from the filename
        disorder_name = filename.split('_')[1].split('.')[0]
        disorder_names.append(disorder_name)

        # Read the file as a DataFrame
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, sep='\t')

        # Add a column with the disorder name
        df['disorder'] = disorder_name

        # Append the DataFrame to the list
        df_list.append(df)

C:\Users\annei\AppData\Local\Temp\ipykernel_10256\538097247.py:19: DtypeWarning: Columns (4,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')
C:\Users\annei\AppData\Local\Temp\ipykernel_10256\538097247.py:19: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')
C:\Users\annei\AppData\Local\Temp\ipykernel_10256\538097247.py:19: DtypeWarning: Columns (4,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')
C:\Users\annei\AppData\Local\Temp\ipykernel_10256\538097247.py:19: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')
C:\Users\annei\AppData\Local\Temp\ipykernel_10256\538097247.py:19: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
 

In [4]:
#select only the columns of interest
new_df_list = []
for df in df_list:
    df = df[['uniqID', 'tissue', 'symbol', 'disorder']]
    new_df_list.append(df)

In [16]:
#get uniqueID and rsID from SNP files
path = 'SNPs'

file_list = os.listdir(path)

files_df = pd.DataFrame()

for file in file_list:
    file_path = os.path.join(path, file)

    df = pd.read_csv(file_path, sep='\t')

    files_df = pd.concat([files_df, df], ignore_index=True)

#Keep only the relevant columns
files_df = files_df[['uniqID', 'rsID','gwasP']]



In [18]:
#map the rsIDs to the SNPid in the df list of gene data
merged_list = []
for df in new_df_list:
    merged_df = pd.merge(df, files_df, on='uniqID', how='left')
    merged_df = merged_df[['uniqID','rsID','gwasP','symbol','tissue','disorder']]
    merged_df = merged_df.rename(columns={'gwasP':'pval_SNP', 'symbol':'gene'})
    merged_list.append(merged_df)


In [19]:
merged_list[13]

,uniqID,rsID,pval_SNP,gene,tissue,disorder
0,1:114075796:C:T,rs10858000,2.776000e-53,DCLRE1B,Nom_Alonso_Pancreatic_islets,LUPUS
1,1:114075796:C:T,rs10858000,3.425000e-03,DCLRE1B,Nom_Alonso_Pancreatic_islets,LUPUS
2,1:114076435:A:C,rs11102648,9.891000e-50,DCLRE1B,Nom_Alonso_Pancreatic_islets,LUPUS
3,1:114076435:A:C,rs11102648,2.567000e-03,DCLRE1B,Nom_Alonso_Pancreatic_islets,LUPUS
4,1:114076546:A:G,rs11102649,1.244000e-47,DCLRE1B,Nom_Alonso_Pancreatic_islets,LUPUS
...,...,...,...,...,...,...
16884410,22:21982249:C:T,rs878825,1.307000e-12,CCDC116,Vagina,LUPUS
16884411,22:21983260:A:G,rs2298429,1.370000e-08,CCDC116,Vagina,LUPUS
16884412,22:21983260:A:G,rs2298429,6.224000e-13,CCDC116,Vagina,LUPUS
16884413,22:21984379:A:G,rs3747093,9.813000e-09,CCDC116,Vagina,LUPUS


In [20]:
def make_pivoted_table(df_list, column1_name, column2_name):
    df = pd.concat(df_list)
    return df.pivot_table(index=column1_name, columns=column2_name, aggfunc=lambda x: 1, fill_value=0)

In [21]:

genes_binary = make_pivoted_table(merged_list, 'gene', 'disorder')
snps_binary = make_pivoted_table(merged_list, 'rsID', 'disorder')

genes_binary.to_csv('genes_binary.txt', sep='\t')
snps_binary.to_csv('snps_binary.txt', sep='\t')

MemoryError: Unable to allocate 911. MiB for an array with shape (119460519,) and data type int64